<a href="https://colab.research.google.com/github/Rahul-131201/English-to-hinglish-machine-translation-using-deep-learning-algorithm-project/blob/main/RNNTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NeuroHack_train.csv to NeuroHack_train.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NeuroHack_test.csv to NeuroHack_test.csv


In [ ]:
test_df = pd.read_csv("NeuroHack_test.csv")

In [ ]:
df = pd.read_csv("NeuroHack_train.csv")

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')
def cleaner():
    for i in range(2566):
        df['English'].iloc[i] = re.sub(CLEANR, '',df['English'].iloc[i])
        words = word_tokenize(df['English'].iloc[i])
        stop = string.punctuation
        t1 = '\n\t'
        stop = stop + t1
        cleaned_word = [w.lower() for w in words if not w in stop]
        df['English'].iloc[i] = cleaned_word
cleaner()

<ipython-input-8-ad90d85ee3e2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['English'].iloc[i] = re.sub(CLEANR, '',df['English'].iloc[i])
<ipython-input-8-ad90d85ee3e2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['English'].iloc[i] = cleaned_word


In [ ]:
def cleaner1():
    for i in range(2566):
        df['Hinglish'].iloc[i] = re.sub(CLEANR, '',df['Hinglish'].iloc[i])
        words = word_tokenize(df['Hinglish'].iloc[i])
        stop = string.punctuation
        t1 = '\n\t'
        stop = stop + t1
        cleaned_word = [w.lower() for w in words if not w in stop]
        df['Hinglish'].iloc[i] = cleaned_word
cleaner1()

<ipython-input-9-b12c91db28ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hinglish'].iloc[i] = re.sub(CLEANR, '',df['Hinglish'].iloc[i])
<ipython-input-9-b12c91db28ce>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hinglish'].iloc[i] = cleaned_word


In [ ]:
df = df.drop(columns = ['Hindi','Average rating','Disagreement'])

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_hinglish_sentences, english_tokenizer, hinglish_tokenizer = preprocess(df['English'], df['Hinglish'])

In [ ]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_hing_sequence_length = preproc_hinglish_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
hing_vocab_size = len(hinglish_tokenizer.word_index)

print("Max English sentence length:", max_english_sequence_length)
print("Max hinglish sentence length:", max_hing_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("hinglish vocabulary size:", hing_vocab_size)

Max English sentence length: 152
Max hinglish sentence length: 148
English vocabulary size: 6715
hinglish vocabulary size: 9554


In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, hing_vocab_size):
    learning_rate = 0.005

    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(hing_vocab_size, activation='softmax')))

    model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, preproc_hinglish_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_hinglish_sentences.shape[-2]))

embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_hinglish_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(hinglish_tokenizer.word_index)+1)

embed_rnn_model.summary()
history = embed_rnn_model.fit(tmp_x, preproc_hinglish_sentences, batch_size=512, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:3])[0], hinglish_tokenizer))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 148, 256)          1719296   
                                                                 
 gru (GRU)                   (None, 148, 256)          394752    
                                                                 
 time_distributed (TimeDistr  (None, 148, 1024)        263168    
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 148, 1024)         0         
                                                                 
 time_distributed_1 (TimeDis  (None, 148, 9555)        9793875   
 tributed)                                                       
                                                                 
Total params: 12,171,091
Trainable params: 12,171,091
No

ValueError: ignored

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, hing_vocab_size):
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003

    # TODO: Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(hing_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_hinglish_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_hinglish_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_hinglish_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(hinglish_tokenizer.word_index)+1)

embed_rnn_model.summary()

history = embed_rnn_model.fit(tmp_x, preproc_hinglish_sentences, batch_size=256, epochs=180, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], hinglish_tokenizer))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 148, 256)          1719296   
                                                                 
 gru_5 (GRU)                 (None, 148, 256)          394752    
                                                                 
 time_distributed_10 (TimeDi  (None, 148, 1024)        263168    
 stributed)                                                      
                                                                 
 dropout_5 (Dropout)         (None, 148, 1024)         0         
                                                                 
 time_distributed_11 (TimeDi  (None, 148, 9555)        9793875   
 stributed)                                                      
                                                                 
Total params: 12,171,091
Trainable params: 12,171,091


ValueError: ignored

In [ ]:
print(logits_to_text(embed_rnn_model.predict(tmp_x[:6])[2], hinglish_tokenizer))

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

NameError: ignored

In [ ]:
print(df.head(10))

                                             English  \
0  [program, module, is, a, file, that, contains,...   
1  [and, to, thamud, we, sent, their, brother, sa...   
2           [and, when, reminded, do, not, remember]   
3                  [you, won, the, ted, prize, 2011]   
4  [he, gone, to, kerodemal, college, of, delhi, ...   
5  [then, we, awakened, them, to, know, which, of...   
6  [really, move, all, photos, and, videos, to, t...   
7  [excavations, made, in, malabar, have, proved,...   
8                [and, leave, them, desolate, waste]   
9  [please, let, me, see, when, you, 're, online,...   

                                            Hinglish  
0  [module, ek, program, hoti, hai, jismen, ya, t...  
1  [aur, hamne, aume, samood, ke, pas, unke, bhae...  
2  [aur, jab, unhen, yad, dilaya, jata, hai, to, ...  
3     [tumhen, २०११, ka, ted, prize, mil, gaya, hai]  
4  [unhonne, bad, science, karne, ke, lie, ye, de...  
5  [phir, hamne, unhen, bheja, taki, maloom, kare... 

In [ ]:
test_df.head()

,English,Hindi
0,"On the other hand, art flourished under Buddhi...",दूसरी ओर बोद्धों के संरक्षण में कला समृद्धशाली...
1,Standard Chartered has brought in 140 million.\n,स्टैंडर्ड चार्टर्ड 140 मिलियन लाया है।\n
2,"During the war years, the railways were under ...","युद्ध के वर्षों के दौरान, रेलवे पर अत्यधिक दबा..."
3,"They said, ‘O Moses! Will you throw first, or ...","वे बोले, ""ऐ मूसा! या तो तुम फेंको या फिर हम पह..."
4,No maintenance charges required to be paid.\n,कोई अनुरक्षण शुल्क देने की जरुरत नहीं है।\n


In [ ]:
def preprocess(x):
    preprocess_x, x_tk = tokenize(x)

    preprocess_x = pad(preprocess_x)

    return preprocess_x, x_tk

preproc_english_sentences,  english_tokenizer = preprocess(test_df['English'])
tmp_test = pad(preproc_english_sentences, preproc_hinglish_sentences.shape[1])
print(logits_to_text(embed_rnn_model.predict(tmp_test[:10])[7], hinglish_tokenizer))
print(preproc_english_sentences)
tmp_test[7]

1/1 [==============================] - 0s 18ms/step
pani ki ki yah ke bhi ki ki rahe hain aur ko milta ke ki ke jo ki aur bhi balki rahe hai 1 se or hai <PAD> hain <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


array([ 10, 380,   8,  22, 173, 174, 381, 382,   6, 383,   1, 384,   2,
       173, 174, 385,   3, 175, 386,   2,  67,  68,   3, 116,   1, 176,
       387, 388,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0], dtype=int32)